In [1]:
import torch
from src.datamodels.pipeline import DatamodelPipeline
from src.datamodels.config import DatamodelConfig
from src.llms import Llama3_1
from src.evaluator import GleuEvaluator


datamodels_path = "../../data/instruction-induction-data/datamodels/proportion_study/210_5"

llama = Llama3_1()

config = DatamodelConfig(
    k = 8,
    num_models= 105,
    datamodels_path = "../../data/instruction-induction-data/datamodels/proportion_study/210_5",
    train_collections_idx = None,
    test_collections_idx = None,
    test_set = None,
    train_set = None,
    instructions= None,
    llm = llama,
    evaluator=GleuEvaluator(),
)


datamodel = DatamodelPipeline(config)



/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


In [2]:
datamodel.load_collections_from_path(test_flag=False)

Collections under processing


In [3]:
dt = torch.load(f"{datamodels_path}/datasets/train/dt_0.pt")

/tmp/ipykernel_1988286/1146515023.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dt = torch.load(f"{datamodels_path}/datasets/train/dt_0.pt")


In [36]:
from torch.utils.data import DataLoader
train = DataLoader(dt, batch_size=8, shuffle=True)

In [37]:
for x, y in train:
    print(x.shape)
    print(y.shape)
    break

torch.Size([8, 41250, 420])
torch.Size([8, 41250])
